# Chapter 1: Basic Configuration & Structure

---

Amazon Titan Image Generator G1 is an image generation model. It generates images from text, and allows users to upload and edit an existing image. This model can generate images from natural language text and can also be used to edit or generate variations for an existing or a generated image. Users can edit an image with a text prompt (without a mask) or parts of an image with an image mask. You can extend the boundaries of an image with outpainting, and fill in an image with inpainting. It can also generate variations of an image based on an optional text prompt.


We will be running Amazon Titan Image Generator G1 on Amazon Bedrock (i.e,. via API and run on AWS-managed servers) for text-to-image-based generation. 


**Lesson:**

To generate an image, Titan Image Generator G1 produces a completely random image (or sample). The model then iterates through a specified number of steps of "denoising" to produce a final sample that is of the desired quality. Prompts must be composed of a term, phrase, or group of words and phrases, separated by commas. The more details provided via the prompt, the higher likelihood the final image will be of desired quality. 


First, we will setup our dependencies.

In [ ]:
%pip install ipython

import json
import boto3
from PIL import Image
import base64
import io
from random import randint
from IPython.display import display

def generate_image(text):
    # Create a new BedrockRuntime client.
    bedrock_runtime = boto3.client(
        "bedrock-runtime",
        region_name="us-east-1",
    )
    
    # Configure the inference parameters.
    inference_params = {
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": text,
            # "negativeText": negative_text,
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "quality": "premium",
            "height": 1024,
            "width": 1024,
            "cfgScale": 8.0,
            "seed": randint(0, 3000),  # Use a random seed
        },
    }
    
    # Invoke the model.
    response = bedrock_runtime.invoke_model(
        modelId="amazon.titan-image-generator-v1", body=json.dumps(inference_params)
    )
    
    # Convert the JSON string to a dictionary.
    response_body = json.loads(response["body"].read())
    
    # Loop through the generated images and display each in the notebook.
    images = response_body["images"]
    for i in range(len(images)):
        image_data = images[i]
        image_bytes = base64.b64decode(image_data)
        image = Image.open(io.BytesIO(image_bytes))
        display(image)

## Basic Prompt Components and Parameters

*text*: A text prompt to generate the image. The basic unit of prompt engineering and the focus of most of our discussion. We will cover techniques in depth in preceding chapters. For now, feel free to experiment with this field

*negative prompts* (optional): similar to the prompt, but here you input what you **don't** want, providing an additional way to control image generation. Uses include removing or tweaking unwanted elements witin images (e.g., trees on a field, waves in the ocean, etc.) or adjusting the image's style (e.g., less blurry or amateur)



*numberOfImages* (optional) – The number of images to generate.

*cfg_scale* (optional): Specifies how strongly the generated image should adhere to the prompt. Use a lower value to introduce more randomness in the generation.

*height* (optional) – The height of the image in pixels. The default value is 1408.

*width* (optional) – The width of the image in pixels. The default value is 1408.

*seed* (optional): Use to control and reproduce results. Determines the initial noise setting. Use the same seed and the same settings as a previous run to allow inference to create a similar image.


## Example:

Let's start with a simple prompt to see what image is generated with the default parameter values

While the above code provides the minimum input necessary for the request to Amazon Bedrock, the below finalizes the request with a text prompt which is sent to the The Amazon Titan Image Generator G1 model to generate an image. 

In [ ]:
text = "beautiful mountain"
generate_image(text)

## Exercises:

With most of the parameters now included in our request to Bedrock, let's experiment.

**Exercise 1.1 - A more beautiful mountain**

Using proper formatting and the included parameters, generate an image of a mountain with the following requirements:
 - use 3 descriptive values / phrases
 - cinematic style
 - ensure the image is relatively the same every time you invoke the model

In [ ]:
text = "A more beautiful mountain with vast scenery and a cartoon style."
generate_image(text)

**Exercise 1.2 - The most beautiful mountain**

Keep tweaking the parameters and prompt to see how you can create an even more high quality, beautiful mountain with the following requirements:

 - add 3 elements to the image (e.g., a river, house, etc.)
 - randomize the model more so the image isn't the same every time and the prompt isn't followed as strictly

In [ ]:
text = "The most beautiful mountain ever with a house, river, and golden shrines."
generate_image(text)

# Chapter 1 - END